In [ ]:
%pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain loguru scikit-learn numpy

`(2) LangSmith`

https://docs.smith.langchain.com/

In [5]:
import os
from google.colab import userdata

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

`(3) API Keys`

In [6]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import json
import shutil
from urllib.parse import urljoin, urlparse
from loguru import logger
from bs4 import BeautifulSoup
import requests
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Scrape a single page
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        logger.error(f"Failed to retrieve {url}: {response.status_code}")
        return None

# Extract links from a page
def extract_links(soup, base_url):
    links = []
    for a_tag in soup.find_all('a', href=True):
        link = a_tag.get('href')
        if link:
            full_link = urljoin(base_url, link) if not urlparse(link).netloc else link
            links.append(full_link)
    logger.info(f"Found {len(links)} links for {base_url}")
    return links

# Extract meaningful content from HTML
def extract_content(html_content, redundant_data):
    soup = BeautifulSoup(html_content, 'html.parser')

    redundant_data_list = [line.strip() for line in redundant_data.split('\n') if line.strip()]

    info = {
        "title": soup.title.string if soup.title else "No title",
        "headers": [header.get_text() for header in soup.find_all(['h1', 'h2', 'h3'])],
        "paragraphs": [para.get_text() for para in soup.find_all('p')],
        "lists": [ul.get_text(separator='\n').strip() for ul in soup.find_all('ul')],
        "tables": [],
        "code_blocks": [],
    }

    # Remove redundant data
    info["headers"] = [header for header in info["headers"] if not any(redundant in header for redundant in redundant_data_list)]
    info["paragraphs"] = [para for para in info["paragraphs"] if not any(redundant in para for redundant in redundant_data_list)]
    info["lists"] = [ul for ul in info["lists"] if not any(redundant in ul for redundant in redundant_data_list)]

    # Extract tables
    for table in soup.find_all('table'):
        table_data = []
        for row in table.find_all('tr'):
            row_data = [cell.get_text().strip() for cell in row.find_all(['td', 'th'])]
            table_data.append(row_data)
        info["tables"].append(table_data)

    # Extract code blocks
    for code in soup.find_all('code', id=True):
        info["code_blocks"].append({
            "id": code['id'],
            "code": code.get_text().strip()
        })

    return info

# Recursive crawling and scraping
def crawl_and_extract(base_url, depth=1, visited=None, redundant_data=""):
    if visited is None:
        visited = set()

    if depth == 0 or base_url in visited:
        return {}

    visited.add(base_url)
    logger.info(f"Crawling: {base_url}")

    html_content = scrape_page(base_url)
    if not html_content:
        return {}

    processed_data = {base_url: extract_content(html_content, redundant_data)}

    soup = BeautifulSoup(html_content, 'html.parser')
    links = extract_links(soup, base_url)
    for link in links:
        processed_data.update(crawl_and_extract(link, depth=depth - 1, visited=visited, redundant_data=redundant_data))

    return processed_data

# Save data to JSON
def save_data(data, output_file):

    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    temp_file = f"{output_file}.tmp"
    with open(temp_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    shutil.move(temp_file, output_file)
    logger.info(f"Data saved to {output_file}")


from langchain.schema import Document
# Load data into LangChain's `WebBaseLoader`
def load_to_webbase_loader(processed_data):
    docs = []
    for url, content in processed_data.items():
        # Modified this line to handle list of lists in tables
        table_text = ["\n".join(str(cell) for cell in row) for row in content["tables"]]
        text = "\n\n".join(
            content["headers"] +
            content["paragraphs"] +
            table_text + # using the modified table text
            [code_block["code"] for code_block in content["code_blocks"]]
        )
        # Create a Document object and append it to the list
        docs.append(Document(page_content=text, metadata={"url": url}))
    return docs

# Main pipeline
def main_pipeline(scrapers, redundant_data, vectorstore_path):
    for scraper in scrapers:
        logger.info(f"Processing {scraper['BASE_URL']}...")

        # Crawl and extract data
        processed_data = crawl_and_extract(scraper["BASE_URL"], depth=2, redundant_data=redundant_data)
        save_data(processed_data, scraper["OUTPUT_FILE"])

        # Load into WebBaseLoader
        logger.info(f"Loading data into WebBaseLoader for {scraper['BASE_URL']}...")
        docs = load_to_webbase_loader(processed_data)

        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=300, chunk_overlap=50
        )
        splits = text_splitter.split_documents(docs)

        print(f"Total splits: {len(splits)}")

        for split in splits:
          print(split)
        # # Index with Chroma
        # logger.info(f"Indexing data for {scraper['BASE_URL']}...")
        # vectorstore = Chroma.from_documents(splits, embedding=OpenAIEmbeddings(), persist_directory=vectorstore_path)
        # # Call persist without any arguments
        # vectorstore.persist()
        # logger.info(f"Data indexed and stored at {vectorstore_path}")

redundant_data = """👋
Introduction
Getting Started
What is Across?
Technical FAQ
Migration Guides
Migration from V2 to V3
Migration to CCTP
Migration Guide for Relayers
Migration Guide for API Users
🔗
Use Cases
Instant Bridging in your Application
Bridge Integration Guide
Multi Chain Bridge UI Guide
Single Chain Bridge UI Guide
Embedded Cross-chain Actions
Cross-chain Actions Integration Guide
Using the Generic Multicaller Handler Contract
Using a Custom Handler Contract
Cross-chain Actions UI Guide
Settle Cross-chain Intents
🧠
Concepts
What are Cross-chain Intents?
Intents Architecture in Across
Intent Lifecycle in Across
Canonical Asset Maximalism
🛠️
Reference
API Reference
SDK Reference
Contracts
Arbitrum (Chain ID: 42161)
Base (Chain ID: 8453)
Blast (Chain ID: 81457)
Ethereum Mainnet (Chain ID: 1)
Linea (Chain ID: 59144)
Lisk (Chain ID: 1135)
Mode (Chain ID: 34443)
Optimism (Chain ID: 10)
Polygon (Chain ID: 137)
Redstone (Chain ID: 690)
Scroll (Chain ID: 534352)
World Chain (Chain ID: 480)
zkSync (Chain ID: 324)
Zora (Chain ID: 7777777)
Sepolia Testnet
Selected Contract Functions
Supported Chains
Fees in the System
Actors in the System
Security Model and Verification
Disputing Root Bundles
Validating Root Bundles
Tracking Events
🔁
Relayers
Running a Relayer
Relayer Exclusivity
📚
Resources
Release Notes
Developer Support
Bug Bounty
Audits
New Chain Requests"""

scrapers = [
    {"BASE_URL": "https://docs.across.to/", "OUTPUT_FILE": "scraper/v3/processed_data_v3.json"},
    {"BASE_URL": "https://docs.across.to/developer-docs", "OUTPUT_FILE": "scraper/v2/processed_data_v2.json"},
    {"BASE_URL": "https://docs.across.to/user-docs/", "OUTPUT_FILE": "scraper/user_docs/processed_data_user_docs.json"}
]

main_pipeline(scrapers, redundant_data, vectorstore_path="vectorstore/knowledge_base")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
import json
import os

def load_processed_json(file_path):
    """Load processed JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def convert_to_documents(data):
    """Convert JSON data into LangChain Document format."""
    documents = []
    for url, content in data.items():
        page_content = (
            f"Title: {content.get('title', '')}\n"
            f"Headers: {', '.join(content.get('headers', []))}\n"
            f"Paragraphs: {' '.join(content.get('paragraphs', []))}\n"
            f"Lists: {' '.join(content.get('lists', []))}\n"
            f"Tables: {content.get('tables', [])}\n"
            f"Code Blocks: {content.get('code_blocks', [])}"
        )
        documents.append(Document(page_content=page_content, metadata={"url": url}))
    return documents

def create_embeddings(input_files, output_dir):
    """Generate embeddings and save them."""
    embeddings = OpenAIEmbeddings()
    for file_path in input_files:
        logger.info(f"Processing {file_path} for embeddings...")
        data = load_processed_json(file_path)
        documents = convert_to_documents(data)

        vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=output_dir)  

        # Save vector store to output directory
        #output_path = os.path.join(output_dir, os.path.basename(file_path).replace(".json", "_embeddings"))  # No longer needed
        vectorstore.persist() #call persist without any arguments
        logger.info(f"Embeddings saved to {output_dir}") #log the output directory

processed_files = [
    "scraper/v3/processed_data_v3.json",
    "scraper/v2/processed_data_v2.json",
    "scraper/user_docs/processed_data_user_docs.json"
]
output_directory = "vector_stores"

os.makedirs(output_directory, exist_ok=True)
create_embeddings(processed_files, output_directory)

In [10]:
def generate_prompt_template(context, query, role="user", detail_level="standard", references=[]):
    is_code_query = detect_query_type(query)
    system_message = get_role_specific_template(role)

    output_instruction = {
        "brief": "Provide a concise response strictly using KB content.",
        "standard": "Provide a clear, context-specific response strictly based on KB content.",
        "detailed": "Provide an in-depth response strictly within the KB context, with comprehensive details and clarifications."
    }.get(detail_level, "Provide a clear, context-specific response strictly based on KB content. Add the source links as well")

    role_instruction = {
        "developer": "Craft a precise code solution or explanation based only on KB content, including inline comments and documentation references." if is_code_query else "Offer a technical explanation rooted in KB specifics, avoiding external examples.",
        "admin": "Provide KB-based configuration guidance or protocol references strictly within the KB scope." if not is_code_query else "Offer a KB-referenced code solution adhering to administrative standards.",
        "user": "Explain concepts or provide relevant context strictly using KB content, limiting the response to KB-verified information." if not is_code_query else "Provide a straightforward code solution derived solely from KB content."
    }.get(role, "Provide a clear, KB-based response without external references.")

    user_message = f"""Context from Knowledge Base:
{context}

User Query:
{query}
{role_instruction}
{output_instruction}
"""
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

def detect_query_type(query):
    """Identify if the query requires a code-based response."""
    code_indicators = [
        'code', 'function', 'implement', 'write', 'program',
        'syntax', 'debug', 'error', 'example', 'script',
        'development', 'api', 'integration'
    ]
    return any(indicator in query.lower() for indicator in code_indicators)

def get_role_specific_template(role):
    """Define a role-specific system message with strict KB adherence and minimal external assumptions."""
    if role == "developer":
        return """You are a highly skilled developer assistant with expertise in Across protocol. Follow these guidelines:
        - Use the knowledge base strictly as the primary source of truth.
        - Avoid assuming knowledge outside of the KB.
        - Provide production-ready code adhering to best practices, with inline comments and structured documentation.
        - Use KB content to address context cues and avoid adding creative assumptions outside the KB."""

    elif role == "admin":
        return """You are an administrative assistant specializing in Across protocol operations and configurations. Your approach should focus on:
        - Ensuring all responses derive strictly from KB content.
        - Providing configuration details, protocol rules, and policy insights only from verified KB information.
        - Minimizing external explanations, focusing on KB-specific instructions and guidelines only."""

    else:  # Default to general user
        return """You are a knowledgeable Across protocol assistant. Respond to user queries as follows:
        - Use the KB as your sole source of truth for information.
        - Break down technical concepts into digestible explanations without adding creative assumptions.
        - Refer only to KB content, ensuring accuracy and adherence to the source material."""



In [ ]:
import openai
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def query_vector_store_with_prompt(vectorstore_path, query_text, role="user", top_k=3, detail_level="standard"):
    """Query the vector store, retrieve relevant documents and generate a prompt template."""
    vectorstore = Chroma(persist_directory=vectorstore_path, embedding_function=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
    results = retriever.get_relevant_documents(query_text)
    context = "\n".join([result.page_content for result in results])
    prompt_template = generate_prompt_template(context, query_text, role, detail_level, references=results)

    print(f"Query: {query_text}\n")
    for i, result in enumerate(results):
        print(f"Result {i + 1}:")
        print(f"Content: {result.page_content}\n")
        print(f"Metadata: {result.metadata}\n")
        print("-" * 80)

    # Return the generated prompt
    return results, prompt_template


# Example usage
query_text = "my tx expired, when I will get my funds back?"
vectorstore_path = "vector_stores"  # Update to your actual vector store path
role = "user"  # Specify the role ("developer", "admin", "user")

# Get the prompt template based on the query and role
results, prompt = query_vector_store_with_prompt(vectorstore_path, query_text, role)


import openai

def generate_openai_response(prompt_template):
    """Generate a response from OpenAI based on the prompt."""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=prompt_template,
            max_tokens=800,
            temperature=0.2
        )

        message = response.choices[0].message.content
        return message

    except Exception as e:
        print(f"Error generating OpenAI response: {e}")
        return None

# Print the generated prompt template
print(json.dumps(prompt, indent=2))  # Pretty print the prompt for debugging purposes
print("Response")
print("-"*50)

from langchain.prompts import ChatPromptTemplate


# Generate OpenAI response based on the prompt
response = generate_openai_response(prompt)
print(response)
print("\n\n\n")

# Format and print references with URLs and similarities below the response
formatted_references = "\n".join(
    [f"Source: {results[0].metadata.get('url')}\n"]
)

if formatted_references:
    print("\nReferences:")
    print(formatted_references)
else:
    print("\nNo URL references found.")